In [ ]:
import numpy as np
import pandas as pd
import scipy
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os
import pprint
import scipy.signal as signal

for i in range(0,1):
    temp_row = cat.iloc[i]
    temp_filename = temp_row.filename
    temp_data_directory = './data/lunar/training/data/S12_GradeA/'
    temp_csv_file = f'data/mars/training/data/XB.ELYSE.02.BHV.2022-02-03HR08_evid0005.csv'
    # temp_csv_file = f'{temp_data_directory}{temp_filename}.csv'
    temp_data_cat = pd.read_csv(temp_csv_file)
    temp_csv_times = np.array(temp_data_cat['rel_time(sec)'].tolist())
    temp_csv_data = np.array(temp_data_cat['velocity(c/s)'].tolist())
    temp_power = temp_csv_data ** 2 / np.sqrt(6.625)
    # temp_mseed_file = f'{temp_data_directory}{temp_filename}.mseed'
    temp_mseed_file = f'data/mars/training/data/XB.ELYSE.02.BHV.2022-02-03HR08_evid0005.mseed'
    
    st = read(temp_mseed_file)
    start_time = temp_row['time_rel(sec)']
    # sos1 = scipy.signal.butter(20, (0.4,1.2), btype='bandstop', analog=False, output='sos', fs=st[0].stats.sampling_rate)
    # sos2 = scipy.signal.butter(20, (0.7,0.8), btype='bandpass', analog=False, output='sos', fs=st[0].stats.sampling_rate)
    sos1 = scipy.signal.butter(20, (0.7,2.4), btype='bandstop', analog=False, output='sos', fs=st[0].stats.sampling_rate)
    sos2 = scipy.signal.butter(20, (1,2.1), btype='bandpass', analog=False, output='sos', fs=st[0].stats.sampling_rate)
    tr = st.traces[0].copy()
    tr_times = tr.times()
    tr_data = tr.data
    result = np.zeros(st[0].stats.npts)
    result_short_avg = np.zeros(st[0].stats.npts)
    result_short_std = np.zeros(st[0].stats.npts)
    result_short_kurtosis = np.zeros(st[0].stats.npts)
    result_short_var = np.zeros(st[0].stats.npts)
    result_short_skew = np.zeros(st[0].stats.npts)
    result_short_rej_short_avg = np.zeros(st[0].stats.npts)
    result_running_diff_var = np.zeros(st[0].stats.npts)
    result_running_diff_var_rej = np.zeros(st[0].stats.npts)
    result_running_diff_norm = np.zeros(st[0].stats.npts)
    result_running_std_diff = np.zeros(st[0].stats.npts)
    index = np.searchsorted(temp_csv_times, 507, side='right')
    window_width = 256
    filtered = signal.sosfilt(sos, tr_data)
    running_avg = []
    rej_running_avg = []
    running_std = []
    triggers = []
    pretriggers = []
    prev_std = 5
    prev_avg = 0
    prev_kurtosis = 1
    # triggers.append(index)
    active = False
    startup = True
    cooldown = False
    result_active = np.zeros(st[0].stats.npts)
    result_cooldown = np.zeros(st[0].stats.npts)
    avg_std_diff =1
    for i in range(0,st[0].stats.npts,window_width):
        temp_reject = signal.sosfilt(sos1, tr_data[i:i+window_width])
        temp_reject = temp_reject ** 2
        rej_short_avg = np.sum(temp_reject) / window_width
        rej_short_std = np.std(temp_reject)
        temp_reject_norm = (temp_reject - rej_short_avg) / rej_short_std
        
        temp = signal.sosfilt(sos2, tr_data[i:i+window_width])
        temp = temp ** 2
        short_avg = np.sum(temp) / window_width
        short_std = np.std(temp)
        temp_norm = (temp - short_avg) / short_std

        
        
        running_avg.append(short_avg)
        rej_running_avg.append(rej_short_avg)
        if (startup):
            result_running_diff_var[i] = 1e-6 * i
            result_running_diff_var_rej[i] = 1e-6 * i
            avg_std_diff = 1
        if (len(running_avg) > 5):
            running_avg.pop(0)
            avg_diff = np.diff(running_avg)
            diff_var = np.std(avg_diff)
            result_running_diff_var[i:] = diff_var
            startup = False
        if (len(rej_running_avg) > 5):
            rej_running_avg.pop(0)
            avg_diff = np.diff(rej_running_avg)
            diff_var = np.std(avg_diff)
            result_running_diff_var_rej[i:] = diff_var
        
        result_short_avg[i:i+window_width] = short_avg
        result_short_rej_short_avg[i:i+window_width] = rej_short_avg
       
        # print(result_running_diff_var[i])
        if (i>0):
            mean_diff_var = np.mean(result_running_diff_var[:i+1])
            std_diff_var = np.std(result_running_diff_var[:i+1])
            mean_diff_var_rej = np.mean(result_running_diff_var_rej[:i+1])
            std_diff_var_rej = np.std(result_running_diff_var_rej[:i+1])
            scaling_factor = (mean_diff_var / mean_diff_var_rej) * (std_diff_var / std_diff_var_rej)
        else:
            scaling_factor = 1
        # print(mean_diff_var_rej)
        # print(scaling_factor)
        running_std_diff = result_running_diff_var[i] - (scaling_factor * result_running_diff_var_rej[i])
        result_running_std_diff[i:] = running_std_diff
        if (not startup):
            avg_std_diff = np.mean(np.abs(result_running_std_diff[:i]))
        std_std_diff = np.std(np.abs(result_running_std_diff[:i]))
        short_vel = np.mean(tr[i:i+window_width])
        avg_short_vel = np.mean(np.abs(tr[:i]))
        std_short_vel = np.std(np.abs(tr[:i]))

        # print(avg_std_diff)

        
        if (any(k > avg_short_vel * 7 for k in np.abs(tr_data[i:i+window_width])) and not active and not cooldown and not startup):
            pretriggers.append(i)
            active = True
        j = 0
        counter = len(pretriggers)
        print(np.sign(running_std_diff) * std_std_diff/avg_std_diff)
        while (j < counter):
            if (i - pretriggers[j] > 1000):
                pretriggers.pop(j)
                counter -= 1
                active = False
            elif (np.sign(running_std_diff) * std_std_diff/avg_std_diff > 2.1):
                triggers.append(pretriggers[j])
                pretriggers.pop(j)
                counter -= 1
                cooldown = True
                active = False
            j += 1
        if (cooldown == True and np.sign(running_std_diff) * std_std_diff/avg_std_diff < 1.5):
            cooldown = False
        result[i:i+window_width] += temp
        result_active[i:] = std_std_diff/avg_std_diff
        result_cooldown[i:] = cooldown

    print((triggers))

    xlim = [0, len(result)]
    # Create subplots
    fig, axs = plt.subplots(5, 1, figsize=(10, 10))
    
    # Plot the first result on the first subplot
    axs[0].plot(result)
    axs[0].set_title('Filtered Result')
    axs[0].set_xlabel('Time [seconds]')
    axs[0].set_xlim(xlim)
    for i in range(len(triggers)):
        arrival_line = axs[0].axvline(x=triggers[i], c='red', label='Calculated')
        arrival_line2 = axs[0].axvline(x=index, c='purple', label='Rel. Arrival')
    axs[0].legend()
    
    # Plot the second result on the second subplot
    axs[1].plot(result_running_std_diff)
    axs[1].set_title('Diff Variance Result')
    axs[1].set_xlim(xlim)
    axs[1].set_xlabel('Time [seconds]')
    for i in range(len(triggers)):
        arrival_line = axs[1].axvline(x=triggers[i], c='red', label='Calculated')
        arrival_line2 = axs[1].axvline(x=index, c='purple', label='Rel. Arrival')
    axs[1].legend()
    
    # Plot the third result on the third subplot
    axs[2].plot(np.abs(tr_data))
    axs[2].set_title('Variance Result')
    axs[2].set_xlim(xlim)
    axs[2].set_xlabel('Time [seconds]')
    for i in range(len(triggers)):
        arrival_line = axs[2].axvline(x=triggers[i], c='red', label='Calculated')
        arrival_line2 = axs[2].axvline(x=index, c='purple', label='Rel. Arrival')
    axs[2].legend()
    
    # Plot the fourth result on the fourth subplot
    axs[3].plot(result_cooldown)
    axs[3].set_title('Skewness Result')
    axs[3].set_xlim(xlim)
    axs[3].set_xlabel('Time [seconds]')
    for i in range(len(triggers)):
        arrival_line = axs[3].axvline(x=triggers[i], c='red', label='Rel. Arrival')
    axs[3].legend()
    
    # Plot the fifth result on the fifth subplot
    axs[4].plot(result_active)
    axs[4].set_title('Kurtosis Result')
    axs[4].set_xlim(xlim)
    axs[4].set_xlabel('Time [seconds]')
    for i in range(len(triggers)):
        arrival_line = axs[4].axvline(x=triggers[i], c='red', label='Rel. Arrival')
    axs[4].legend()
    
    # Make the plot layout tight
    plt.tight_layout()
    plt.show()